In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
solver = po.SolverFactory("gurobi")
model = pe.ConcreteModel()

### Sets Lookup Cell

In [3]:
factories = {
    "Ijmuiden",
    "Segal",
    "South Wales",
}

customers = {
    "Bochum",
    "Boenen",
    "Dortmund",
    "Gelsenkirchen",
    "Hagen",
    "Iserlohn",
    "Neuss",
    "Schwerte",
}

rebar_types = {"A", "B", "C"}
long_bar_types = {"Type 1", "Type 2"}


### Parameter Lookup cell

In [4]:
production_capacity = {
    "Ijmuiden": 12,
    "Segal": 10,
    "South Wales": 28,
}

factory_storage_capacity = {
    "Ijmuiden": 30,
    "Segal": 15,
    "South Wales": 60,
}

customer_storage_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5,
}

vehicle_capacity = 25 # in tonnes

factory_fixed_cost = {
    "Ijmuiden": 400,
    "Segal": 500,
    "South Wales": 250,
}

factory_variable_cost = {
    "Ijmuiden": 10,
    "Segal": 5,
    "South Wales": 2,
}

transport_fixed_cost = {
    "Ijmuiden": 130,
    "Segal": 150,
    "South Wales": 100,
}

uncut_bar_weight = {  # in tonnes
    "Type 1": 0.1803,
    "Type 2": 0.2404,
}

rebar_weight = {  # in tonnes
    "A": 0.0481,
    "B": 0.0721,
    "C": 0.0841,
}

patterns = { # for reference
    ("Type 1", 1): (3, 0, 0),
    ("Type 1", 2): (2, 1, 0),
    ("Type 1", 3): (0, 2, 0),
    ("Type 1", 4): (0, 1, 1),
    ("Type 1", 5): (0, 0, 2),
    ("Type 1", 6): (2, 0, 1),
    ("Type 2", 1): (5, 0, 0),
    ("Type 2", 2): (3, 1, 0),
    ("Type 2", 3): (3, 0, 1),
    ("Type 2", 4): (2, 2, 0),
    ("Type 2", 5): (1, 1, 1),
    ("Type 2", 6): (1, 0, 2),
    ("Type 2", 7): (1, 2, 0),
    ("Type 2", 8): (0, 1, 2),
    ("Type 2", 9): (0, 2, 1),
}

rebars_from_pattern = {
    ("Type 1", 1, "A"): 3,
    ("Type 1", 1, "B"): 0,
    ("Type 1", 1, "C"): 0,
    ("Type 1", 2, "A"): 2,
    ("Type 1", 2, "B"): 1,
    ("Type 1", 2, "C"): 0,
    ("Type 1", 3, "A"): 0,
    ("Type 1", 3, "B"): 2,
    ("Type 1", 3, "C"): 0,
    ("Type 1", 4, "A"): 0,
    ("Type 1", 4, "B"): 1,
    ("Type 1", 4, "C"): 1,
    ("Type 1", 5, "A"): 0,
    ("Type 1", 5, "B"): 0,
    ("Type 1", 5, "C"): 2,
    ("Type 1", 6, "A"): 2,
    ("Type 1", 6, "B"): 0,
    ("Type 1", 6, "C"): 1,
    ("Type 2", 1, "A"): 5,
    ("Type 2", 1, "B"): 0,
    ("Type 2", 1, "C"): 0,
    ("Type 2", 2, "A"): 3,
    ("Type 2", 2, "B"): 1,
    ("Type 2", 2, "C"): 0,
    ("Type 2", 3, "A"): 3,
    ("Type 2", 3, "B"): 0,
    ("Type 2", 3, "C"): 1,
    ("Type 2", 4, "A"): 2,
    ("Type 2", 4, "B"): 2,
    ("Type 2", 4, "C"): 0,
    ("Type 2", 5, "A"): 1,
    ("Type 2", 5, "B"): 1,
    ("Type 2", 5, "C"): 1,
    ("Type 2", 6, "A"): 1,
    ("Type 2", 6, "B"): 0,
    ("Type 2", 6, "C"): 2,
    ("Type 2", 7, "A"): 1,
    ("Type 2", 7, "B"): 2,
    ("Type 2", 7, "C"): 0,
    ("Type 2", 8, "A"): 0,
    ("Type 2", 8, "B"): 1,
    ("Type 2", 8, "C"): 2,
    ("Type 2", 9, "A"): 0,
    ("Type 2", 9, "B"): 2,
    ("Type 2", 9, "C"): 1,
}

demand = {
    (1, "Bochum", "A"): 2,
    (1, "Boenen", "A"): 4,
    (1, "Dortmund", "A"): 2,
    (1, "Gelsenkirchen", "A"): 5,
    (1, "Hagen", "A"): 19,
    (1, "Iserlohn", "A"): 13,
    (1, "Neuss", "A"): 20,
    (1, "Schwerte", "A"): 4,

    (2, "Bochum", "A"):6,
    (2, "Boenen", "A"):8,
    (2, "Dortmund", "A"):7,
    (2, "Gelsenkirchen", "A"):5,
    (2, "Hagen", "A"):23,
    (2, "Iserlohn", "A"):19,
    (2, "Neuss", "A"):16 ,
    (2, "Schwerte", "A"):5,

    (3, "Bochum", "A"):5,
    (3, "Boenen", "A"):5,
    (3, "Dortmund", "A"):6,
    (3, "Gelsenkirchen", "A"):5,
    (3, "Hagen", "A"):25,
    (3, "Iserlohn", "A"):17,
    (3, "Neuss", "A"):14,
    (3, "Schwerte", "A"):3,

    (4, "Bochum", "A"):3,
    (4, "Boenen", "A"):10,
    (4, "Dortmund", "A"):5,
    (4, "Gelsenkirchen", "A"):5,
    (4, "Hagen", "A"):16,
    (4, "Iserlohn", "A"):14,
    (4, "Neuss", "A"):26,
    (4, "Schwerte", "A"):4,


    (1, "Bochum", "B"): 4,
    (1, "Boenen", "B"): 5,
    (1, "Dortmund", "B"): 4,
    (1, "Gelsenkirchen", "B"): 9,
    (1, "Hagen", "B"): 15,
    (1, "Iserlohn", "B"): 22,
    (1, "Neuss", "B"): 12,
    (1, "Schwerte", "B"): 2,

    (2, "Bochum", "B"):5,
    (2, "Boenen", "B"):8,
    (2, "Dortmund", "B"):5,
    (2, "Gelsenkirchen", "B"):10,
    (2, "Hagen", "B"):33,
    (2, "Iserlohn", "B"):26,
    (2, "Neuss", "B"):23,
    (2, "Schwerte", "B"):8 ,

    (3, "Bochum", "B"):7,
    (3, "Boenen", "B"):12,
    (3, "Dortmund", "B"):8,
    (3, "Gelsenkirchen", "B"):6,
    (3, "Hagen", "B"):31,
    (3, "Iserlohn", "B"):20,
    (3, "Neuss", "B"):30,
    (3, "Schwerte", "B"):2,

    (4, "Bochum", "B"):8,
    (4, "Boenen", "B"):13,
    (4, "Dortmund", "B"):10,
    (4, "Gelsenkirchen", "B"):6,
    (4, "Hagen", "B"):33,
    (4, "Iserlohn", "B"):27,
    (4, "Neuss", "B"):30,
    (4, "Schwerte", "B"):6,

    (1, "Bochum", "C"): 6,
    (1, "Boenen", "C"): 6,
    (1, "Dortmund", "C"): 7,
    (1, "Gelsenkirchen", "C"): 10,
    (1, "Hagen", "C"): 12,
    (1, "Iserlohn", "C"): 14,
    (1, "Neuss", "C"): 22,
    (1, "Schwerte", "C"): 5,

    (2, "Bochum", "C"): 7,
    (2, "Boenen", "C"): 10,
    (2, "Dortmund", "C"): 6,
    (2, "Gelsenkirchen", "C"): 9,
    (2, "Hagen", "C"): 35,
    (2, "Iserlohn", "C"): 25,
    (2, "Neuss", "C"): 32,
    (2, "Schwerte", "C"): 6,

    (3, "Bochum", "C"): 7,
    (3, "Boenen", "C"): 15,
    (3, "Dortmund", "C"): 4,
    (3, "Gelsenkirchen", "C"): 9,
    (3, "Hagen", "C"): 33,
    (3, "Iserlohn", "C"): 23,
    (3, "Neuss", "C"): 31,
    (3, "Schwerte", "C"): 7,

    (4, "Bochum", "C"): 7,
    (4, "Boenen", "C"): 12,
    (4, "Dortmund", "C"): 12,
    (4, "Gelsenkirchen", "C"): 10,
    (4, "Hagen", "C"): 38,
    (4, "Iserlohn", "C"): 24,
    (4, "Neuss", "C"): 31,
    (4, "Schwerte", "C"): 2,
}

transport_distances = {
    ('Ijmuiden', 'Ijmuiden'): 0,
    ('Ijmuiden', 'Segal'): 284,
    ('Ijmuiden', 'South Wales'): 826,
    ('Ijmuiden', 'Bochum'): 250,
    ('Ijmuiden', 'Boenen'): 282,
    ('Ijmuiden', 'Dortmund'): 266,
    ('Ijmuiden', 'Gelsenkirchen'): 234,
    ('Ijmuiden', 'Hagen'): 289,
    ('Ijmuiden', 'Iserlohn'): 299,
    ('Ijmuiden', 'Neuss'): 259,
    ('Ijmuiden', 'Schwerte'): 279,
    
    ('Segal', 'Ijmuiden'): 284,
    ('Segal', 'Segal'): 0,
    ('Segal', 'South Wales'): 750,
    ('Segal', 'Bochum'): 203,
    ('Segal', 'Boenen'): 242,
    ('Segal', 'Dortmund'): 222,
    ('Segal', 'Gelsenkirchen'): 198,
    ('Segal', 'Hagen'): 206,
    ('Segal', 'Iserlohn'): 226,
    ('Segal', 'Neuss'): 140,
    ('Segal', 'Schwerte'): 216,
    
    ('South Wales', 'Ijmuiden'): 826,
    ('South Wales', 'Segal'): 750,
    ('South Wales', 'South Wales'): 0,
    ('South Wales', 'Bochum'): 866,
    ('South Wales', 'Boenen'): 914,
    ('South Wales', 'Dortmund'): 885,
    ('South Wales', 'Gelsenkirchen'): 859,
    ('South Wales', 'Hagen'): 903,
    ('South Wales', 'Iserlohn'): 913,
    ('South Wales', 'Neuss'): 843,
    ('South Wales', 'Schwerte'): 901,
    
    ('Bochum', 'Ijmuiden'): 250,
    ('Bochum', 'Segal'): 203,
    ('Bochum', 'South Wales'): 866,
    ('Bochum', 'Bochum'): 0,
    ('Bochum', 'Boenen'): 55,
    ('Bochum', 'Dortmund'): 21,
    ('Bochum', 'Gelsenkirchen'): 19,
    ('Bochum', 'Hagen'): 41,
    ('Bochum', 'Iserlohn'): 51,
    ('Bochum', 'Neuss'): 56,
    ('Bochum', 'Schwerte'): 39,
    
    ('Boenen', 'Ijmuiden'): 282,
    ('Boenen', 'Segal'): 242,
    ('Boenen', 'South Wales'): 914,
    ('Boenen', 'Bochum'): 55,
    ('Boenen', 'Boenen'): 0,
    ('Boenen', 'Dortmund'): 34,
    ('Boenen', 'Gelsenkirchen'): 56,
    ('Boenen', 'Hagen'): 44,
    ('Boenen', 'Iserlohn'): 54,
    ('Boenen', 'Neuss'): 102,
    ('Boenen', 'Schwerte'): 32,

    ('Dortmund', 'Ijmuiden'): 266,
    ('Dortmund', 'Segal'): 222,
    ('Dortmund', 'South Wales'): 885,
    ('Dortmund', 'Bochum'): 21,
    ('Dortmund', 'Boenen'): 34,
    ('Dortmund', 'Dortmund'): 0,
    ('Dortmund', 'Gelsenkirchen'): 34,
    ('Dortmund', 'Hagen'): 21,
    ('Dortmund', 'Iserlohn'): 36,
    ('Dortmund', 'Neuss'): 75,
    ('Dortmund', 'Schwerte'): 15,

    ('Gelsenkirchen', 'Ijmuiden'): 234,
    ('Gelsenkirchen', 'Segal'): 198,
    ('Gelsenkirchen', 'South Wales'): 859,
    ('Gelsenkirchen', 'Bochum'): 19,
    ('Gelsenkirchen', 'Boenen'): 56,
    ('Gelsenkirchen', 'Dortmund'): 34,
    ('Gelsenkirchen', 'Gelsenkirchen'): 0,
    ('Gelsenkirchen', 'Hagen'): 56,
    ('Gelsenkirchen', 'Iserlohn'): 66,
    ('Gelsenkirchen', 'Neuss'): 62,
    ('Gelsenkirchen', 'Schwerte'): 54,
    
    ('Hagen', 'Ijmuiden'): 289,
    ('Hagen', 'Segal'): 206,
    ('Hagen', 'South Wales'): 903,
    ('Hagen', 'Bochum'): 41,
    ('Hagen', 'Boenen'): 44,
    ('Hagen', 'Dortmund'): 21,
    ('Hagen', 'Gelsenkirchen'): 56,
    ('Hagen', 'Hagen'): 0,
    ('Hagen', 'Iserlohn'): 20,
    ('Hagen', 'Neuss'): 66,
    ('Hagen', 'Schwerte'): 14,
    
    ('Iserlohn', 'Ijmuiden'): 299,
    ('Iserlohn', 'Segal'): 226,
    ('Iserlohn', 'South Wales'): 913,
    ('Iserlohn', 'Bochum'): 51,
    ('Iserlohn', 'Boenen'): 54,
    ('Iserlohn', 'Dortmund'): 36,
    ('Iserlohn', 'Gelsenkirchen'): 66,
    ('Iserlohn', 'Hagen'): 20,
    ('Iserlohn', 'Iserlohn'): 0,
    ('Iserlohn', 'Neuss'): 85,
    ('Iserlohn', 'Schwerte'): 15 ,

    ('Neuss', 'Ijmuiden'): 259,
    ('Neuss', 'Segal'): 140,
    ('Neuss', 'South Wales'): 843,
    ('Neuss', 'Bochum'): 56,
    ('Neuss', 'Boenen'): 102,
    ('Neuss', 'Dortmund'): 75,
    ('Neuss', 'Gelsenkirchen'): 62,
    ('Neuss', 'Hagen'): 66,
    ('Neuss', 'Iserlohn'): 85,
    ('Neuss', 'Neuss'): 0,
    ('Neuss', 'Schwerte'): 75,
    
    ('Schwerte', 'Ijmuiden'): 279,
    ('Schwerte', 'Segal'): 216,
    ('Schwerte', 'South Wales'): 901,
    ('Schwerte', 'Bochum'): 39,
    ('Schwerte', 'Boenen'): 32,
    ('Schwerte', 'Dortmund'): 15,
    ('Schwerte', 'Gelsenkirchen'): 54,
    ('Schwerte', 'Hagen'): 14,
    ('Schwerte', 'Iserlohn'): 15,
    ('Schwerte', 'Neuss'): 75,
    ('Schwerte', 'Schwerte'): 0,
}



### Pyomo Model | Sets

In [5]:
model.Factories = pe.Set(initialize=factories, ordered=False)
model.Bars = pe.Set(initialize=long_bar_types, ordered=False)
model.Rebars = pe.Set(initialize=rebar_types, ordered=False)
model.Customers = pe.Set(initialize=customers, ordered=False)

model.Cities = pe.Set(initialize=(factories.union(customers)), ordered = False)

model.Periods = pe.RangeSet(1, 4)
model.PatternsType1 = pe.RangeSet(1, 6)
model.PatternsType2 = pe.RangeSet(1, 9)

### Pyomo Model | Variables

In [6]:
# First, we would like to know which factory is used for producing bars
model.factory_active = pe.Var(
    model.Periods,
    model.Factories,
    domain=pe.Binary,
)

# Next, we track what type of long bar is produced by each factory
model.production_bar = pe.Var(
    model.Periods,
    model.Factories,
    model.Bars,
    domain=pe.NonNegativeIntegers,
)

# Afterwards, we want to know how the factories cut their bars
model.rebars_cut_type1 = pe.Var(
    model.Periods,
    model.Factories,
    model.PatternsType1,
    domain=pe.NonNegativeIntegers,
)

model.rebars_cut_type2 = pe.Var(
    model.Periods,
    model.Factories,
    model.PatternsType2,
    domain=pe.NonNegativeIntegers,
)

# Two additional variables to chech what is stored by the factory/customeres
model.factory_storage = pe.Var(
    model.Periods,
    model.Factories,
    model.Bars,
    domain=pe.NonNegativeIntegers,
)

model.customer_storage = pe.Var(
    model.Periods,
    model.Customers,
    model.Rebars,
    domain=pe.NonNegativeIntegers,
)

model.allow_for_delivery = pe.Var(
    model.Periods,
    model.Factories, # vehicles fleet
    domain=pe.Binary
)

model.ships = pe.Var(
    model.Periods,
    model.Factories, # vehicles fleet
    model.Cities,
    model.Cities,
    domain=pe.Binary,
)

model.carries = pe.Var(
    model.Periods,
    model.Factories, # vehicle
    model.Cities,
    model.Rebars,
    domain=pe.NonNegativeIntegers,
)

### Pyomo Model | Objective

In [7]:
# Cost for having a factory open
cost_factory_open = sum(model.factory_active[period, factory] * factory_fixed_cost[factory] for factory in model.Factories for period in model.Periods)

# variable cost for producing bars
cost_producing_bars = sum(model.production_bar[period, factory, bar] * factory_variable_cost[factory] for bar in model.Bars for factory in model.Factories for period in model.Periods)

# Variable cost for shipping
cost_per_km_tonnes = sum(model.ships[period, factory, node_i, node_j] * transport_distances[node_i, node_j] * 3 for factory in model.Factories for node_j in model.Cities for node_i in model.Cities for period in model.Periods)

### Total objective
objective = cost_factory_open + cost_producing_bars + cost_per_km_tonnes
model.obj = pe.Objective(sense=pe.minimize, expr=objective)

### Pyomo Model | Constraints

In [8]:
### each factory cannot produce more than capacity

model.max_production_capacity = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        expr = sum(model.production_bar[period, factory, bar] * uncut_bar_weight[bar] for bar in model.Bars) <= production_capacity[factory]
        
        # print(expr) # Uncomment to see constraint
        
        # Add to model
        model.max_production_capacity.add(expr=expr)

In [9]:
### You can only produce bars when the factory is open

model.open_for_production = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        expr = sum(model.production_bar[period, factory, bar] for bar in model.Bars) <= model.factory_active[period, factory] * 10000
        
        # print(expr) # Uncomment to see constraint
        
        # Add to model
        model.open_for_production.add(expr=expr)

In [10]:
## You save all the bars you didn't cut, and can use them in the next period.

model.running_inventory = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
            if period == 1:
                expr_1 = model.factory_storage[period, factory, 'Type 1'] == 0
                expr_2 = model.factory_storage[period, factory, 'Type 2'] == 0
                
                # print(expr_1) # Uncomment to see constraint	
                # print(expr_2) # Uncomment to see constraint	
                
                model.running_inventory.add(expr=expr_1)
                model.running_inventory.add(expr=expr_2)
            
            else:
                expr_1 = (model.factory_storage[period, factory, 'Type 1'] == model.factory_storage[period - 1, factory, 'Type 1'] + model.production_bar[period - 1, factory, 'Type 1'] - sum(model.rebars_cut_type1[period - 1, factory, pattern] for pattern in model.PatternsType1) )
                expr_2 = (model.factory_storage[period, factory, 'Type 2'] == model.factory_storage[period - 1, factory, 'Type 2'] + model.production_bar[period - 1, factory, 'Type 2'] - sum(model.rebars_cut_type2[period - 1, factory, pattern] for pattern in model.PatternsType2) )
                
                # print(expr) # Uncomment to see constraint	
                # print(expr) # Uncomment to see constraint	
                
                model.running_inventory.add(expr=expr_1)
                model.running_inventory.add(expr=expr_2)

In [11]:
### You cannot store more than capacity in the factory

model.max_storage_capacity = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories: 
        expr = sum(model.factory_storage[period, factory, bar] * uncut_bar_weight[bar] for bar in model.Bars) <= factory_storage_capacity[factory]
        
        # print(expr) # Uncomment to see constraint
  
        model.max_storage_capacity.add(expr=expr)

In [12]:
### You can only cut bars when you produced them or have them on storage

model.max_cutting_bars = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
     
        expr_1 = (sum(model.rebars_cut_type1[period, factory, pattern] for pattern in model.PatternsType1) ) <= model.production_bar[period, factory, 'Type 1']  + model.factory_storage[period, factory, 'Type 1']
        expr_2 = (sum(model.rebars_cut_type2[period, factory, pattern] for pattern in model.PatternsType2) ) <= model.production_bar[period, factory, 'Type 2']  + model.factory_storage[period, factory, 'Type 2']
        
        # print(expr) # Uncomment to see constraint
        # print(expr) # Uncomment to see constraint
    
        model.max_cutting_bars.add(expr=expr_1)
        model.max_cutting_bars.add(expr=expr_2)

In [13]:
# you can only use your own vehicle to ship to customers
model.cant_use_other_factory_vehicle = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        expr = sum(model.ships[period, factory, factory_i, j] for factory_i in model.Factories if (factory != factory_i) for j in model.Cities) == 0
        
        # print(expr)  # Uncomment to see constraint

        model.cant_use_other_factory_vehicle.add(expr=expr)

In [14]:
# Each customer can only be served by at most one factory
model.served_by_one = pe.ConstraintList()

for period in model.Periods:
    for customer in model.Customers:
    
        expr = sum(model.ships[period, factory, i, customer] for factory in model.Factories for i in model.Cities) <= 1
    
        # print(expr)  # Uncomment to see constraint

        model.served_by_one.add(expr=expr)

In [15]:
# Each vehicle can be used at most once
model.vehicle_used_once = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories: 
        expr = sum(model.ships[period, factory, factory, j] for j in model.Customers) <= 1
    
        # print(expr)  # Uncomment to see constraint
    
        model.vehicle_used_once.add(expr=expr)
        

In [16]:
# to carry you must first ship
model.to_serve_is_to_ship = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        for customer in model.Customers:
            expr = sum(model.carries[period, factory, customer, rebar] for rebar in model.Rebars) <= sum(model.ships[period, factory, i, customer] for i in model.Cities) * 10000
            
            # print(expr)  # Uncomment to see constraint
    
            model.to_serve_is_to_ship.add(expr=expr)            
                

In [17]:
# Flow balance for each vehicle

model.flow_balance = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        for node_j in model.Cities:
            expr = sum(model.ships[period, factory, node_i, node_j] for node_i in model.Cities) == sum(model.ships[period, factory, node_j, node_k] for node_k in model.Cities)
            
            # print(expr)  # Uncomment to see constraint

            model.flow_balance.add(expr=expr)    

In [18]:
# Subtours elimination constraints
model.sub_tour_elimination = pe.ConstraintList()

for period in model.Periods:
    for node_i in model.Customers:
        for node_j in model.Customers:
            if node_i != node_j and node_j not in factories:
                for factory in model.Factories:
                        transport = sum(model.carries[period, factory, node_i, rebar] * rebar_weight[rebar] for rebar in model.Rebars)
                        c_demand = sum(demand[period, node_j, rebar]*rebar_weight[rebar] for rebar in model.Rebars)
                        expr =  + transport + c_demand <= sum(model.carries[period, factory, node_j, rebar]*rebar_weight[rebar] for rebar in model.Rebars) + 25 * (1 - model.ships[period, factory, node_i, node_j])
                        
                        # print(expr)  # Uncomment to see constraint
                        
                        model.sub_tour_elimination.add(expr)

In [19]:
# Don't allow loops
model.no_loops = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        for node_i in model.Cities:
            
            expr = model.ships[period, factory, node_i, node_i ] == 0
            
            model.no_loops.add(expr)

In [20]:
# Can't carry more than vehicles capacity
model.vehicle_capacity = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        expr = sum(model.carries[period, factory, node_i, rebar]*rebar_weight[rebar] for rebar in model.Rebars for node_i in model.Cities) <= 25
        
        # print(expr)
        
        model.vehicle_capacity.add(expr)
            

In [21]:
### A shipment cannot carry more than is produced or available

model.max_carry_capacity = pe.ConstraintList()

for period in model.Periods:
    for factory in model.Factories:
        for rebar in model.Rebars:
            
            available_rebars_from_type1 = sum(model.rebars_cut_type1[period, factory, pattern] * rebars_from_pattern['Type 1', pattern, rebar] for pattern in model.PatternsType1) + model.factory_storage[period, factory, 'Type 1']
            available_rebars_from_type2 = sum(model.rebars_cut_type2[period, factory, pattern] * rebars_from_pattern['Type 2', pattern, rebar] for pattern in model.PatternsType2) + model.factory_storage[period, factory, 'Type 2']
            
            expr = sum(model.carries[period, factory, nodes, rebar] for nodes in model.Cities) <= available_rebars_from_type1 + available_rebars_from_type2
            
            # print(expr)  # Uncomment to see constraint
            
            model.max_carry_capacity.add(expr=expr)

In [22]:
### You save all the bars you immediately use for filling up the demand, and can use them in the next period.

model.storing_rebars = pe.ConstraintList()

for period in model.Periods:
    for customer in model.Customers:
        for rebar in model.Rebars:
            if period == 1:
                expr = model.customer_storage[period, customer, rebar] == 0
                
                # print(expr) # Uncomment to see constraint	
                
                model.storing_rebars.add(expr=expr)
            
            else:
                expr = sum(model.carries[period - 1, factory, customer, rebar] for factory in model.Factories) + model.customer_storage[period - 1, customer, rebar] - demand[period - 1, customer, rebar] == model.customer_storage[period, customer, rebar]
                
                # print(expr) # Uncomment to see constraint	
                
                model.storing_rebars.add(expr=expr)


In [23]:
### You cannot store more than capacity in the warehouse

model.max_warehouse_capacity = pe.ConstraintList()

for period in model.Periods:
    for customer in model.Customers: 
        expr = sum(model.customer_storage[period, customer, rebar] * rebar_weight[rebar] for rebar in model.Rebars) <= customer_storage_capacity[customer]
        
        # print(expr) # Uncomment to see constraint
  
        model.max_warehouse_capacity.add(expr=expr)

In [24]:
# Each demand should be met

model.meet_demand = pe.ConstraintList()

for period in model.Periods:
    for customer in model.Customers:
        for rebar in model.Rebars:
            expr = demand[period, customer, rebar] <= sum(model.carries[period, factory, customer, rebar] for factory in model.Factories) + model.customer_storage[period, customer, rebar]
        
            # print(expr)  # Uncomment to see constraint
            
            model.meet_demand.add(expr=expr)

### Pyomo Model | Solver

In [25]:
result = solver.solve(model, tee= True, options = {'TimeLimit':1600})

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-04
Read LP format model from file /var/folders/xk/nwlnsjsx3c321c24ljrms4mc0000gn/T/tmpn5jk4pll.pyomo.lp
Reading time = 0.01 seconds
x1: 1444 rows, 2184 columns, 12795 nonzeros
Set parameter TimeLimit to value 1600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1444 rows, 2184 columns and 12795 nonzeros
Model fingerprint: 0x41590e65
Variable types: 0 continuous, 2184 integer (1464 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+04]
  Objective range  [2e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 301 rows and 738 columns
Presolve time: 0.01s
Presolved: 1143 rows, 1446 columns, 10088 nonzeros
Variable types: 0 continuous, 1446 integer (876 binary)

Root relaxation: objective 9.

### Pyomo Model | Results

In [26]:
print(f"objective value is: {pe.value(model.obj)}")

##### Active Factories

# Which factories where open?
print('\nOpen factories:')
for period in model.Periods:
    for factory in model.Factories:
        if pe.value(model.factory_active[period, factory]) > 0:
            print(f"{period, factory} : {pe.value(model.factory_active[period, factory])}")

##### Producing

# how many bars did we produce?
print('\nHow many bars produced:')
for period in model.Periods:
    for factory in model.Factories:
        for bar in model.Bars:
            if pe.value(model.production_bar[period, factory, bar]) > 0:
                print(f"{period, factory, bar} : {pe.value(model.production_bar[period, factory, bar])}")
 
 
 ##### Storing
 
 # how many bars did we store?
print('\nHow many bars did we store:')
for period in model.Periods:
    for factory in model.Factories:
        for bar in model.Bars:
            if pe.value(model.factory_storage[period, factory, bar]) > 0:
                print(f"{period, factory, bar} : {pe.value(model.factory_storage[period, factory, bar])}")
 
 
 ##### Cutting bars
 
 # how many patterns did we cut?
print('\npatterns from steel bar type 1:')
for period in model.Periods:
    for factory in model.Factories:
        for pattern in model.PatternsType1:
            if pe.value(model.rebars_cut_type1[period, factory, pattern]) > 0:
                print(f"{period, factory, pattern} : {pe.value(model.rebars_cut_type1[period, factory, pattern])}")       

 # how many patterns did we cut?
print('\npatterns from steel bar type 2:')
for period in model.Periods:
    for factory in model.Factories:
        for pattern in model.PatternsType2:
            if pe.value(model.rebars_cut_type2[period, factory, pattern]) > 0:
                print(f"{period, factory, pattern} : {pe.value(model.rebars_cut_type2[period, factory, pattern])}")     
        
        
##### Delivery


 # Which factory served which customer?
print('\nWhich customer was served by which factory:')
for period in model.Periods:
    for factory in model.Factories:
        for node_i in model.Cities:
            for node_j in model.Cities:
                if pe.value(model.ships[period, factory, node_i, node_j]) > 0:
                    print(f"{period, factory, node_i, node_j} : {pe.value(model.ships[period, factory, node_i, node_j])}")     


 # How much was carried by the vehicle?
print('\nhow much did each shipment carry:')
for period in model.Periods:
    for factory in model.Factories:
        for nodes in model.Cities:
            for rebar in model.Rebars:
                if pe.value(model.carries[period, factory, nodes, rebar]) > 0:
                    print(f"{period, factory, nodes, rebar} : {pe.value(model.carries[period, factory, nodes, rebar])}")     


# ##### Storing

# How much did the customer store                 
print('\nHow much did the customer store:')
for period in model.Periods:
    for customer in model.Customers:
        for rebar in model.Rebars:
            if pe.value(model.customer_storage[period, customer, rebar]) > 0:
                print(f"{period, customer, rebar} : {pe.value(model.customer_storage[period, customer, rebar])}")     

objective value is: 15724.0

Open factories:
(1, 'South Wales') : 1.0
(1, 'Segal') : 1.0
(1, 'Ijmuiden') : 1.0
(2, 'Segal') : 1.0
(2, 'Ijmuiden') : 1.0

How many bars produced:
(1, 'South Wales', 'Type 2') : 104.0
(1, 'Segal', 'Type 2') : 4.0
(1, 'Segal', 'Type 1') : 50.0
(1, 'Ijmuiden', 'Type 2') : 49.0
(1, 'Ijmuiden', 'Type 1') : 1.0
(2, 'Segal', 'Type 2') : 36.0
(2, 'Ijmuiden', 'Type 2') : 48.0

How many bars did we store:
(2, 'Segal', 'Type 2') : 4.0
(2, 'Segal', 'Type 1') : 50.0
(2, 'Ijmuiden', 'Type 2') : 6.0
(2, 'Ijmuiden', 'Type 1') : 1.0
(3, 'Segal', 'Type 2') : 1.0
(3, 'Segal', 'Type 1') : 50.0

patterns from steel bar type 1:
(2, 'Ijmuiden', 6) : 1.0
(3, 'Segal', 2) : 1.0
(3, 'Segal', 3) : 5.0
(3, 'Segal', 5) : 15.0
(3, 'Segal', 6) : 29.0

patterns from steel bar type 2:
(1, 'South Wales', 1) : 5.0
(1, 'South Wales', 2) : 1.0
(1, 'South Wales', 4) : 44.0
(1, 'South Wales', 8) : 54.0
(1, 'Ijmuiden', 4) : 27.0
(1, 'Ijmuiden', 8) : 16.0
(2, 'Segal', 4) : 1.0
(2, 'Segal', 8) : 3

### Pyomo Model | Diagnostics

In [27]:
print(f"\nstatus: {result.solver.status}, condition: {result.solver.termination_condition}")


status: aborted, condition: maxTimeLimit
